# Stopword들을 찾고 제거하거나 1개로 통일하자

## 양 옆에 공백을 두고 있지만 의미없는 단어

In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
from collections import defaultdict

In [2]:
def make_dataframe(path: str) -> pd.DataFrame:
    """
    Read a json file and return a pandas DataFrame.

    Parameters:
    path (str): Path to the json file.

    Returns:
    pd.DataFrame: DataFrame of the json file.
    """
    # Read the json file
    with open(path, 'r') as file:
        data = json.load(file)

    # Create a DataFrame
    # columns = ['id', 'conversation', 'subject_keyword', 'output']
    df = pd.DataFrame(data)
    df['conversation'] = df['input'].apply(lambda x: x['conversation'])
    df['subject_keyword'] = df['input'].apply(lambda x: x['subject_keyword'])

    # Drop the 'input' column
    df.drop('input', axis=1, inplace=True)

    # Speakers in the conversation
    df['speakers'] = df['conversation'].apply(lambda turns: list(set(turn['speaker'] for turn in turns)))

    # Reorder the columns
    df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output']]

    return df

In [3]:
train_df = make_dataframe('../resource/data/일상대화요약_train.json')
dev_df = make_dataframe('../resource/data/일상대화요약_dev.json')
test_df = make_dataframe('../resource/data/일상대화요약_test.json')

In [8]:
# find the stopwords
def find_stopwords(df: pd.DataFrame, pattern) -> set:
    """
    Find the stopwords in the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame of the json file.

    Returns:
    set: Set of stopwords.
    """
    # Find the stopwords
    stopwords = defaultdict(int)

    for idx, row in df.iterrows():
        for turn in row['conversation']:
            utterance = turn['utterance']

            # Find the stopwords and add them to stopwords
            stopwords_list = re.findall(pattern, utterance)
            for stopword in stopwords_list:
                stopwords[stopword] += 1

    # Make a Series of stopwords
    stopwords = pd.Series(stopwords)

    print(f"Number of stopwords: {len(stopwords)}")

    return stopwords

In [32]:
stopwords = find_stopwords(train_df, pattern = r'(?:\s+[가-힣]{1}\s+)')

In [33]:
stopwords.sort_values(ascending=False, inplace=True)

In [34]:
len(stopwords)

728

In [36]:
stopwords[:50]

 거     5283
 뭐     4345
 그     4264
 좀     2492
 안     2460
 또     2343
 게     2236
 한     1729
 때     1435
 막     1396
 잘     1281
 다     1269
 더     1052
 수      881
 어      801
 못      768
 할      755
 해      699
 걸      668
 것      627
 이      601
 아      589
 건      530
 내      509
 나      434
 난      349
 가      325
 될      317
 꼭      310
 참      305
 왜      301
 갈      290
 딱      285
 제      273
 데      270
 저      249
 몇      242
 본      241
 두      229
 큰      220
 볼      212
 사      170
 너      162
 줄      154
 된      154
 네      135
 뭘      135
 살      133
 날      133
 원      132
dtype: int64

In [41]:
import re

# 테스트 문자열들
texts = [
    "그래 그래서",   # 매칭됨
    "아니 아니야",   # 매칭됨
    "하지 하지만",   # 매칭됨
    "먹먹 먹",       # 매칭되지 않음
    "그래 그래",     # 매칭되지 않음
    "응 응해",       # 매칭됨
]

# 정규 표현식 패턴
pattern = r'\b([가-힣]+)\s+\1[가-힣]*\b'

# 각 문자열에 대해 패턴과 일치하는지 확인
for text in texts:
    match = re.search(pattern, text)
    if match:
        print(f"Match found in '{text}':", match.group())
    else:
        print(f"No match found in '{text}'")

Match found in '그래 그래서': 그래 그래서
Match found in '아니 아니야': 아니 아니야
Match found in '하지 하지만': 하지 하지만
No match found in '먹먹 먹'
Match found in '그래 그래': 그래 그래
Match found in '응 응해': 응 응해


In [96]:
sw = find_stopwords(train_df, pattern = r'\b([가-힣]+)\s+\1([가-힣]+\b)')

In [108]:
sw.sort_values(ascending=False, inplace=True)
sw[10:20]

아  니      4
하  고      4
그  거      4
이  게      4
그  냥      4
   거를     4
어  떻게     4
그  렇게     4
   러니까    4
아  직      3
dtype: int64

In [94]:
re.findall(r'\b([가-힣]+)\s+\1([가-힣]+)\b' ,'그래 그래서 애니메이션이 진격의 거인이었 거인이었거든요.')

[('그래', '서'), ('거인이었', '거든요')]

In [89]:
re.sub(r'\b([가-힣]+)\s+\1([가-힣]*)\b', r'\1','그래 그래서 애니메이션이 진격의 거인이었 거인이었거든요.')

'그래 애니메이션이 진격의 거인이었.'

In [71]:
import re

# 테스트 문자열들
texts = [
    "그래 그래서",   # 매칭됨
    "아니 아니야",   # 매칭됨
    "하지 하지만",   # 매칭됨
    "먹먹 먹",       # 매칭되지 않음
    "그래 그래",     # 매칭되지 않음
    "응 응해",       # 매칭됨
]

# 정규 표현식 패턴
pattern = r'\b([가-힣]+)\s+\1([가-힣]+)\b'

# 각 문자열에 대해 패턴과 일치하는지 확인하고 치환
for text in texts:
    result = re.sub(pattern, r'\1\2', text)
    print(f"Original: '{text}' => Modified: '{result}'")

Original: '그래 그래서' => Modified: '그래서'
Original: '아니 아니야' => Modified: '아니야'
Original: '하지 하지만' => Modified: '하지만'
Original: '먹먹 먹' => Modified: '먹먹 먹'
Original: '그래 그래' => Modified: '그래 그래'
Original: '응 응해' => Modified: '응해'


## X를 포함하는 단어들

In [24]:
find_stopwords(test_df, pattern = r'\b[가-힣a-zA-Z]*[xX][가-힣a-zA-Z]*\b')

Number of stopwords: 59


xxx          4
xx로          1
x           28
x는           2
제x           1
다니x          1
즐겁더라고x       1
xx케          1
x케           1
xx          15
요즘x          1
어디xx         1
요x           1
이x           3
마마마마x        1
근x           1
싶x           1
생각xx         1
알았x          1
같x           2
운x을          1
그x           2
같으xx요        1
그래x          1
왜x면          1
그xx          1
x이지만         1
그래갖고서x       1
먹x           1
그x고          1
끊x고          1
x다음에         1
xx서          6
한x           1
x번           1
xx으로         1
올라x고         1
x보적으로        1
아이x          1
xxxxx습니다     1
x거           1
xx도          1
x짜           1
무난했xx        1
xxxx         2
x데           4
그x까          1
같xx          1
xx게          1
x시           1
해x고          1
꼽으라x         1
x니까          4
x게           1
xx까          1
싶xx          1
지난xx         1
xx튼          1
x까           1
dtype: int64